In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import skew

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek

import pickle

#for Reggresion
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV




#for classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from  xgboost import XGBRFClassifier
from sklearn.metrics import accuracy_score,auc,confusion_matrix,classification_report


#For hyperparameter tuning
from sklearn.model_selection import GridSearchCV







: 

In [ ]:
data=pd.read_csv("C:/Users/admin pc/Downloads/Copper_Set.xlsx - Result 1.csv")

: 

In [ ]:
data.head(3)

: 

In [ ]:
data.shape

: 

In [ ]:
data.info()

: 

In [ ]:
data.describe()

: 

In [ ]:
data.keys()

: 

In [ ]:
data.isnull().sum()

: 

In [ ]:
ax=sns.heatmap(data.isnull())
ax.set_title("Before cleaning process")

: 

In [ ]:
data_1=data.copy()

: 

In [ ]:
#Meterial reference feature has lot of null values(77919)so drop the column,and drop id feature also
data_1.drop(["material_ref","id"],axis=1,inplace=True)

: 

In [ ]:
#preprocessing of data 


: 

In [ ]:
#we see that pandas has loaded some of the numerical features as object type.We will explicitly convert those columns into numerical

: 

In [ ]:
def data_convert():
        #"quantity tons",contain a non numerical value we have to replace that with NaN
        data_1["quantity tons"]=pd.to_numeric(data_1["quantity tons"],errors="coerce")
        #To convert catogorical "item type" feature to numerical using label encoding
        label_encoder=preprocessing.LabelEncoder()
        data_1['item type']=label_encoder.fit_transform(data_1['item type'])
        #Similarly convert "status" catogorical feature to numerical (machine learning model only take numerical data)
        label_encoder2=preprocessing.LabelEncoder()
        data_1['status']=label_encoder2.fit_transform(data_1['status'])



        #Convert float data type to date format by converting non date value to NaN
        data_1["item_date"]=pd.to_datetime(data_1["item_date"],format="%Y%m%d",errors="coerce")
        data_1["delivery date"]=pd.to_datetime(data_1["delivery date"],format="%Y%m%d",errors="coerce")
        return data_1
new_data=data_convert()

: 

In [ ]:
#Now let's check encoded data
data['item type'].unique()

: 

In [ ]:
new_data.isnull().sum()

: 

In [ ]:
#plot hisogram to check distribution of data 
features=["country", "status","customer","application","thickness","selling_price","quantity tons"]
new_data[features].hist(figsize=(10,8))

: 

: 

In [ ]:
#in this plots we can see that distribution in features is skewed so we can fill the missing value with median
def missing_fill():
    new_data["country"].fillna(new_data["country"].median(),inplace=True)
    new_data["customer"].fillna(new_data["customer"].median(),inplace=True)
    new_data["application"].fillna(new_data["application"].median(),inplace=True)
    new_data["thickness"].fillna(new_data["thickness"].median(),inplace=True)
    new_data["selling_price"].fillna(new_data["selling_price"].median(),inplace=True)
    new_data["quantity tons"].fillna(new_data["quantity tons"].median(),inplace=True)
    #Handling missing data in catogorical feature 
    new_data["status"].fillna(new_data["status"].mode().iloc[0],inplace=True)
    #we can handle the missing data in datetime feature by mode
    new_data["item_date"].fillna(new_data["item_date"].mode().iloc[0],inplace=True)
    new_data["delivery date"].fillna(new_data["delivery date"].mode().iloc[0],inplace=True)
    return new_data
cleaned_data=missing_fill()

: 

In [ ]:
#Let's check after cleaning all the missing values
cleaned_data.isnull().sum() 

: 

In [ ]:
ax=sns.heatmap(cleaned_data.isnull())
ax.set_title("After cleaning process")

: 

In [ ]:
#plt.figure(figsize = (10,8))
#sns.heatmap(data_1.corr(), cmap = 'coolwarm',annot=True,fmt=".2f")

: 

In [ ]:
#feature scaling

: 

In [ ]:

def plot(df, column):
    plt.figure(figsize=(14,5))
    plt.subplot(1,3,1)
    sns.boxplot(data=df, x=column)
    plt.title(f'Box Plot for {column}')
    print(f'{column}')
    print("skew: " ,skew(df[column]))

    
    plt.subplot(1,3,2)
    sns.histplot(data=df, x=column, kde=True, bins=50)
    plt.title(f'Distribution Plot for {column}')

    plt.subplot(1,3,3)
    sns.violinplot(data=df, x=column)
    plt.title(f'Violin Plot for {column}')
    plt.show()

: 

In [ ]:
columns=[]
for col in cleaned_data.drop(["delivery date","item_date",'status'],axis=1):
    columns.append(col)
    
for i in columns :
    plot(cleaned_data, i)

: 

In [ ]:
#from above plot we can see that almost all the features are skwed,among them 'quantity tons', 'customer,'thickness', 'selling_price' very highly skewed,
#So applay log transformation to change skewd data to normal

: 

In [ ]:
l=[]
for i in  cleaned_data["quantity tons"].values:
    if i<=0:
        i=np.nan
        l.append(i)
    else:
        l.append(i)  
cleaned_data["quantity tons"]=l

: 

In [ ]:
list2=[]
for item2 in  cleaned_data["selling_price"].values:
    if item2<=0:
        item2=np.nan
        list2.append(item2)
    else:
        list2.append(item2)  
cleaned_data["selling_price"]=list2

: 

In [ ]:
cleaned_data.isnull().sum()

: 

In [ ]:
cleaned_data.dropna(inplace=True)

: 

In [ ]:
cleaned_data_new=cleaned_data.copy()

: 

In [ ]:
#To remove skewness i have used log method
cleaned_data["quantity_tons_log"]=np.log(cleaned_data["quantity tons"])
cleaned_data["selling_price_log"]=np.log(cleaned_data["selling_price"])
cleaned_data["customer_log"]=np.log(cleaned_data["customer"])
cleaned_data["thickness_log"]=np.log(cleaned_data["thickness"])
cleaned_data["product_ref_log"]=np.log(cleaned_data["product_ref"])



: 

In [ ]:
cleaned_data.drop(["quantity tons","customer","selling_price","thickness","product_ref"],axis=1,inplace=True)

: 

In [ ]:
cleaned_data.head(1)

: 

In [ ]:
#This plot shows the distribution after remove the skewness
def plot(df, column):
    plt.figure(figsize=(14,5))
    plt.subplot(1,3,1)
    sns.boxplot(data=df, x=column)
    plt.title(f'Box Plot for {column}')


    
    plt.subplot(1,3,2)
    sns.histplot(data=df, x=column, kde=True, bins=50)
    plt.title(f'Distribution Plot for {column}')

    plt.subplot(1,3,3)
    sns.violinplot(data=df, x=column)
    plt.title(f'Violin Plot for {column}')
    plt.show()

: 

In [ ]:

for i in ["customer_log","thickness_log","product_ref_log","quantity_tons_log","selling_price_log"] :
    plot(cleaned_data,i)

: 

In [ ]:
#Now i am going to remove outlier,Using statistical method (IQR)

: 

In [ ]:
def out(df,column):
    q1=df[column].quantile(.25)
    q3=df[column].quantile(.75)
    IQR=q1-q3
    uppr=q1+(1.5*IQR)
    lwr=q1-(1.5*IQR)
    df[column]=df[column].clip(uppr,lwr)
    return df
for item1 in ["customer_log","thickness_log","product_ref_log","quantity_tons_log","selling_price_log"]: 
                                                   df1=out(cleaned_data,item1)


: 

In [ ]:
#Let's check the  plot After removing the oulier
plt.rcParams["figure.figsize"] = [8.50, 6.50]
plt.rcParams["figure.autolayout"] = True

ax1=df1[["customer_log","thickness_log","product_ref_log","quantity_tons_log","selling_price_log"]].plot(kind="box",title="After removing outliers")
plt.show()

: 

In [ ]:
#lt's check correlation between the features
plt.figure(figsize=(12,8))
sns.heatmap(cleaned_data.corr(),annot=True,cmap="coolwarm")

: 

In [ ]:
#There are high correlations between selling_price and item date,delivery date compare to other features

: 

In [ ]:
cleaned_data1=cleaned_data.copy()

: 

In [ ]:
#let's convert date time into year month day saparatly
cleaned_data['item_date_year']=cleaned_data["item_date"].dt.year
cleaned_data['item_date_month']=cleaned_data["item_date"].dt.month
cleaned_data['item_date_day']=cleaned_data["item_date"].dt.day
cleaned_data['delivery_date_year']=cleaned_data["delivery date"].dt.year
cleaned_data['delivery_date_month']=cleaned_data["delivery date"].dt.month
cleaned_data['delivery_date_day']=cleaned_data["delivery date"].dt.day


: 

In [ ]:
cleaned_data.drop(["item_date","delivery date"],axis=1,inplace=True)


: 

In [ ]:
#NOw Normalize he data using minimax scalar
#scalar=MinMaxScaler()
#new_data1=scalar.fit_transform(cleaned_data)
#new_data1=pd.DataFrame(new_data1,columns=["country","status","item type","application","width","quantity_tons_log","selling_price_log","customer_log","thickness_log","product_ref_log","item_date_year","item_date_month","item_date_day","delivery_date_year","delivery_date_month","delivery_date_day"])

: 

In [ ]:
#Here completed the preprocessing of given data,Now going to select the model

: 

In [ ]:
cleaned_data.head(1)

: 

In [ ]:
x=cleaned_data.drop(["selling_price_log"],axis=1)
y=cleaned_data["selling_price_log"]
scalar=StandardScaler()
x=scalar.fit_transform(x)

: 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2)

     

: 

In [ ]:
models=[("RandomForestRegressor",RandomForestRegressor()),
        ("GradientBoostingRegressor",GradientBoostingRegressor()),
        ("DecisionTreeRegressor",DecisionTreeRegressor()),
        ("ExtraTreesRegressor",ExtraTreesRegressor()),
        ("XGBRegressor",XGBRegressor())]

: 

In [ ]:
for name,model in models:
    print(name,":")
    model.fit(x_train,y_train)
    y_predict=model.predict(x_test)
    print("r2_score :\n",r2_score(y_test,y_predict))
    print()
    print("mean_absolute_error :\n",mean_absolute_error(y_test,y_predict))
    print()
    print("mean_squared_error :\n",mean_squared_error(y_test,y_predict))
    print()

: 

In [ ]:
#From above evaluation  result we can see that RandomForestRegressor is the best algorithm for moodel with less overfitting and underfitting

: 

In [ ]:
#Train the model using RandomForestRegressor algorithm 

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2)
Reggression_model=RandomForestRegressor()
Reggression_model.fit(x_train,y_train)
y_predict=Reggression_model.predict(x_test)
print("R2_test",r2_score(y_test,y_predict))

: 

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

: 

In [ ]:
grid_search = GridSearchCV(estimator = Reggression_model, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

: 

In [ ]:
grid_search.fit(x_train, y_train)

: 

In [ ]:
best_grid = grid_search.best_estimator_
print("R2_test",r2_score(y_test,y_predict))

: 

: 

In [ ]:
import pickle
#serailising
with open("Regression_model.pk1","wb") as f1:
    pickle.dump(Reggression_model,f1)

with open("Scalar.pk1","wb") as f2:
    pickle.dump(scalar,f2)

: 

In [ ]:
#de-serialing
with open("Regression_model.pk1","rb") as f3:
    R_model=pickle.load(f3)

with open("Scalar.pk1","rb") as f4:
    scalar=pickle.load(f4)


: 

In [ ]:
cleaned_data.head(1)

: 

In [ ]:
cleaned_data.iloc[265]

: 

In [ ]:
test=np.array([[25.000000,7.000000,5.000000,26.000000,1062.000000,5.238603,17.223530,0.405465,21.236568,2021.000000,4.000000,1.000,2021.00,7.00,1.00]])
test=scalar.transform(test)
y_p=R_model.predict(test)

: 

In [ ]:
cleaned_data.iloc[336]

: 

In [ ]:
print(y_p)

: 

In [ ]:
#we have trained and stored the reggression model ,NOw let"s move into classification model selection and training

#Have to predict the catogarical feature status

: 

In [ ]:
new_cleaned_data=cleaned_data[(cleaned_data["status"]==1)|(cleaned_data["status"]==7)]
new_cleaned_data.status = new_cleaned_data.status.map({1: 0, 7: 1})


: 

In [ ]:
new_cleaned_data.status.value_counts()

: 

In [ ]:
#Here we can see that the data is not balanced,we should balance the data otherwise model will learn from majority
#So balance the data using SMOTE((Synthetic Minority Over-sampling Technique)

X=new_cleaned_data.drop("status",axis=1)
Y=new_cleaned_data["status"]


: 

In [ ]:
Y.value_counts()

: 

In [ ]:
X_new,Y_new= SMOTETomek().fit_resample(X,Y)

: 

In [ ]:
Y_new.value_counts()

: 

In [ ]:
scalar_=StandardScaler()
X_new=scalar_.fit_transform(X_new)
X_train,X_test,Y_train,Y_test=train_test_split(X_new,Y_new,test_size=.2)

: 

In [ ]:
#algorithms for classification 
Models=[("RandomForestClassifier",RandomForestClassifier()),
        ("DecisionTreeClassifier",DecisionTreeClassifier()),
        ("ExtraTreesClassifier",ExtraTreesClassifier()),
        ("GradientBoostingClassifier",GradientBoostingClassifier()),
        ("XGBRFClassifier",XGBRFClassifier())]

        

: 

In [ ]:
for name1,Model1 in Models:
            print(name1 ,":")
            Model1.fit(X_train,Y_train)
            y_pred=Model1.predict(X_test)
            print("accuracy_score_test : ",accuracy_score(Y_test,y_pred))
            print("\n")
            print("classification_report : ",classification_report(Y_test,y_pred))
            print("confusion_matrix : ",confusion_matrix(Y_test,y_pred))


: 

In [ ]:
#The algorith ExtraTreesClassifier is best for model because it has high accuracy compare to other

: 

In [ ]:
#Let's train the data using  ExtraTreesClassifier  algorthm
classification_model=ExtraTreesClassifier()
classification_model.fit(X_train,Y_train)
Y_predict=classification_model.predict(X_test)
 


: 

In [ ]:

#param_grid = { 'n_estimators': [25, 50, 100, 150], 
   ## 'max_features': ['sqrt', 'log2', None], 
   ## 'max_depth': [3, 6, 9], 
  #3#  'max_leaf_nodes': [3, 6, 9]} 


#grid_search=GridSearchCV(ExtraTreesClassifier,param_grid=param_grid)
#grid_search.fit(x_train,y_train)
#print(grid_search.best_estimator_)

#Update model
#Reggression_model=RandomForestRegressor('max_depth':,'max_features':,'max_leaf_nodes':,'n_estimators':)
#Reggression_model.fit(x_train,y_train)
#y_predict=Reggression_model.predict(x_test)
#print("R2_test",r2_score(y_test,y_predict))


: 

In [ ]:

#serailising
with open("classification_model.pk1","wb") as f:
    pickle.dump(classification_model,f)

with open("Scalar_C.pk1","wb") as f_:
    pickle.dump(scalar_,f_)




: 

In [ ]:
#de-serialing
with open("classification_model.pk1","rb") as f4:
    C_model=pickle.load(f4)


with open("Scalar_C.pk1","rb") as f4_:
    Scalar=pickle.load(f4_)

: 

In [ ]:
data.iloc[171]

: 

In [ ]:
test=np.array([[39.000000,5.000000,56.000000,1155.000000,3.249582,7.021976,17.223269,0.182322,21.235312,2021.000000,4.000000,1.000000,2021.000000,8.000000,1.000000]])

y_p=C_model.predict(test)

: 

In [ ]:
print(y_p)

: 

In [ ]:
#we have trined both classification and regresssion models for catogorical and continues variable prediction
#Now have to predict the depended variable "selling price"(continues) and status(catogory) by getting input from user using streamlit application

: 

In [ ]:
data.iloc[254]

: 